<a href="https://colab.research.google.com/github/jhyoo78/jhyoo78/blob/main/2_6_Q_learning__SFC__Deployment__20200418_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
''' ******************* SFC placement and deployment Program by jhyoo, 2020.04.18 *********************'''
'''                                                                                                    '''
'''  Edge 내, 각 서버의 cpu_no와 mem_no는 임의 설정가능하지만 8개 이상을 권고함                                           '''
'''  시험용 SFC의 길이는 3 ~ 7개가 적절함. 특별한 경우(8개 이상은 비현실적)에는 reward 값의 재조정이 필요함     '''
'''                                                                                                    '''
''' ***************************************************************************************************'''

# 구현에 사용할 패키지 임포트하기
import numpy as np
import ast
import copy

'''--------------------전체 서버에 대한 상태 값 설정 및 행렬 생성------------------ '''
'''-------------- 향후 NFV management/orchestrarion System에서 monitoring 한 값으로 대체해야 함. 본 학습에서는 임의로 설정 가능함----------'''
# edge 서버 자원 정의
edge_no = 3            # Edge switch의 수 (=시험할 Edge의 수)
tor_no = 3             # edge swtch 별 tor swtch의 수
svr_no = 3             # TOR에 연결된 서버의 갯수
cpu_no = 32            # 서버 내 cpu core 수
mem_no = 64            # 서버 내 메모리 량(GBytes)

local_svr_no = edge_no*tor_no*svr_no              # 전체 서버의 수
self_svr_no = local_svr_no/edge_no                # Edge 내 서버의 수 --> 나눗셈에 의해 dtype == float 로 바뀌므로 주의, 아래도...
neigh_svr_no = local_svr_no - self_svr_no         # 이웃하는 Edge(전화국)내 서버의 수,  하나의 Edge에 대한 이웃 Edge의 수는 1 개 이상일 수 있음

# Data Center의 서버 자원 정의
dc_edge_no = 2            # Edge switch의 수 (DC는 규모가 크므로 2개로 가정함)
dc_tor_no = 3             # edge swtch 별 tor swtch의 수
dc_svr_no = 10             # TOR에 연결된 서버의 갯수
dc_cpu_no = 48            # 서버 내 vcpu 수
dc_mem_no = 96            # 서버 내 메모리 량

dc_local_svr_no = dc_edge_no*dc_tor_no*dc_svr_no   # DC 내 서버의 수


total_svr_no =local_svr_no + dc_local_svr_no     # 전체 서버의 수

print("local_svr_no=", local_svr_no)   # [0, 1, 2, 3, 4, 5......17]
print("self_svr_no=", self_svr_no)     # [ 6, 7, 8, 9]
print("neigh_svr_no=", neigh_svr_no)   # [0,1, 2, 3, 4, 5, 10...17]
print("total_svr_no=", total_svr_no)

vnf_type=np.array([[1,1],    # vnf0은 cpu 1 개, mem 1 GB 사용
                  [1, 2],    # vnf1은 cpu 1 개, mem 2 GB 사용....
                  [2, 2],
                  [2, 4],
                  [3, 3],
                  [3, 6],
                  [4, 4],
                  [4, 8]])    # VNF 7은 cpu 4개, mem 8GB 사용

vnf_no=len(vnf_type)

svr_states = np.zeros((local_svr_no, 2)) + [cpu_no, mem_no]    # ---> 호출 문제로 함수 내부로 이전
print("svr_states=",svr_states)

dc_svr_states = np.zeros((dc_local_svr_no, 2)) + [dc_cpu_no, dc_mem_no]    # ---> 호출 문제로 함수 내부로 이전
#print("dc_svr_states=",dc_svr_states)

total_svr_states = np.append(svr_states, dc_svr_states, axis=0)    #np.append()로 2개의 array를 통합함
#print("total_svr_states=", total_svr_states)



local_svr_no= 27
self_svr_no= 9.0
neigh_svr_no= 18.0
total_svr_no= 87
svr_states= [[32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]
 [32. 64.]]


In [ ]:
## 정책 theta_0 의 초기값 생성
''' theta_0 를 0으로 초기화하여 생성하고, svr_states 행렬의 서버 별 자원 상태와 vnf_type 행렬의 자원 요구량을 비교하여
자원이 충분하면 theta_0의 각원소를 1로 설정하고 부족하면 설치 불가하므로 0으로 설정한다.'''

'''state는 VNF의 자원요구량에 의해 상태 변화를 야기하는 것으로...
action은 가용 자원이 충분한 어떤 서버를 선택하여 설치하는 것으로 정의했다'''

theta_0 = np.zeros((int(vnf_no),int(total_svr_no)))     # 행= VNF_type 의 갯수,  열 = 전체 서버 수
#print("theta_0=", theta_0)

for a in range(total_svr_no):
    for b in range(vnf_no):
        if total_svr_states[a][0] >= vnf_type[b][0] and total_svr_states[a][1] >= vnf_type[b][1]:  # 특정 서버의 자원이 각각의 VNF_type 설치에 충분하면 값을 1로
            theta_0[b][a] = 1
        else:                                                                          # 부족하면 0으로...
            theta_0[b][a] = np.nan
print("theta_0=", theta_0)

'''---------------------------------------------------------------------------------'''


theta_0= [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

'---------------------------------------------------------------------------------'

In [ ]:
# 정책 파라미터 theta_0을 무작위 행동 정책 pi로 변환하는 함수
def simple_convert_into_pi_from_theta(theta):
    '''단순 비율 계산'''

    [m, n] = theta.shape  # theta의 행렬 크기를 구함
    pi = np.zeros((m, n))
    for i in range(0, m):
        pi[i, :] = theta[i, :] / np.nansum(theta[i, :])  # 비율 계산
        #print(pi[i, :])
    pi = np.nan_to_num(pi)  # nan을 0으로 변환
    #print(pi)
    return pi

# 무작위 행동정책 pi_0을 계산하여 이후 학습에 사용함
pi_0 = simple_convert_into_pi_from_theta(theta_0)
print("pi_0=", pi_0)

# 행동가치 함수 Q의 초기화 상태

[a, b] = theta_0.shape  # 열과 행의 갯수를 변수 a, b에 저장
Q = np.random.rand(a, b) * theta_0     # Uniform 분포로 0.0~1.0 값의 array를 생성함. theta_0를 곱한 이유는 nan을 반영하기 위한 것
'''----------------------------------------------------------'''
print("Q=", Q)   #  Q 테이블 값 출력

'''------------------------------------------------------------------------'''
# Q러닝 알고리즘으로 행동가치 함수 Q를 수정---- a_next를 사용하지 않는다.

def Q_learning(s, a, r, s_next, Q, eta, gamma):

    if s_next == 8888:  # 목표 지점에 도달한 경우
        #print("End of a SFC deployment")
        Q[s, a] = Q[s, a] + eta * (r - Q[s, a])
        #print("Q_1=", Q[s, a])
    else:
        Q[s, a] = Q[s, a] + eta * (r + gamma * np.nanmax(Q[s_next,: ]) - Q[s, a])
        #print("Q_2=", Q[s, a])

    return Q

''' ----  np.random.choice()에서 종종 발생하는 에러("probabilities do not sum to 1") 대책으로 Kahansum을 사용함 '''
def kahansum(input):
    summ = c = 0
    for num in input:
        y = num - c
        t = summ + y
        c = (t - summ) - y
        summ = t
    return summ

pi_0= [[0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425 0.01149425 0.01149425 0.01149425
  0.01149425 0.01149425 0.01149425]
 [0.01

In [ ]:
''' 아래의 svr_loc()함수는 Latency 계산을 위해서 이전 action(p_act)과 현재 action이 자국, 인접국, DC 중 어디에 속하는 것이지를 찾아준다
즉, s_a_history 리스트에서 이전 action과 현재 action을 추출하여 svr_loc()함수를 호출하면,
함수는 np.where()를 이용하여 이 속하는 <Edge 번호, TOR 번호, 서버 번호>를 찾고 이를 이용하여 hop 수를 계산하도록 한다 '''

def svr_loc(edge_seq_id, p_act,action):                 # edge_seq_id는 sfc_req.txt에서 추출한 edge 번호, p_act는  바로 이전에 취한 action의 번호임(previous action)
    '''---- 참조정보----------------------------'''
    total_svr_list = np.arange(total_svr_no, dtype=int)
    local_svr_list = np.arange(local_svr_no, dtype=int)
    #print("전체 서버 번호=", total_svr_list)
    #print("로컬  서버 번호=", local_svr_list)

    self_svr_list= np.arange(local_svr_no//edge_no*(edge_seq_id - 1),local_svr_no//edge_no * edge_seq_id, dtype='i')  # 자국의 좌우에 인접국이 여럿 있는 경우도 있으므로 이를 고려하여.... 자국 서버 번호를 추출함.
    #print("자국 서버 번호=", self_svr_list)
    #print("dtype=",self_svr_list.dtype)

    neigh_svr_list= np.delete(local_svr_list, self_svr_list)
    #print(" 인접국 서버 번호=", neigh_svr_list)
    dc_svr_list = np.arange(local_svr_no, total_svr_no)
    #print(" DC 서버 번호=", dc_svr_list)

    '''-----자국, 인접국 및 DC에 대한 리스트를 생성-------------------'''

    self = self_svr_list.reshape(tor_no, svr_no)
    neigh = neigh_svr_list.reshape(edge_no-1, tor_no, svr_no)    # 3차원으로 수정, neigh는 전체 스위치(edge_no) 수에서 self가 사용하는 것을 제외하므로
    dc = dc_svr_list.reshape(dc_edge_no,dc_tor_no, dc_svr_no)
    #print("self=", self)
    #print("neigh=", neigh)
    #print("dc=", dc)
    ''' ----------- np.where()를 이용하여 edge, tor, server 번호를 추출함------------- '''
    p_self_ret= np.where(self == p_act)                          # self 리스트에서 p_act(예:9) 에 해당하는 인덱스를  반환함.
    #print("p_self_ret=", p_self_ret)                            # --> (array([0]), array([1])) 등으로 출력됨
    #print("p_self_ret=", p_self_ret[0], p_self_ret[1])

    lc_p_self_ret = list(p_self_ret)                              # lc는 list conversion을 의미함, np.where에서 return된 tuple을 list로 변경하여 빈 값을 -1로 변경해줌(오류 방지 용).
                                                                  # [array([0]), array([1])]
    '''  # 출력 예: p_self_ret= (array([1], dtype=int64), array([0], dtype=int64)) 또는 p_self_ret= (array([], dtype=int64), array([], dtype=int64))'''

    if len(lc_p_self_ret[0])==0 and len(lc_p_self_ret[1])==0:  # 출력 예에서 값이 값이 없는 경우에도 연산할 수 있도록 -1을 추가함(위의 출력 예에서 후자(빈 array[]에 해당하는 경우)
        lc_p_self_ret[0]= -1
        lc_p_self_ret[1]= -1
    #print("lc_p_self_ret=", tuple(lc_p_self_ret))
    #print("lc_p_self_ret=",lc_p_self_ret[0],lc_p_self_ret[1])

    p_neigh_ret= np.where(neigh == p_act)
    #print("p_neigh_ret=", p_neigh_ret)
    #print("p_neigh_ret=", p_neigh_ret[0], p_neigh_ret[1], p_neigh_ret[2])
    lc_p_neigh_ret=list(p_neigh_ret)                             # list conversion
    if len(lc_p_neigh_ret[0])==0 and len(lc_p_neigh_ret[1])==0 and len(lc_p_neigh_ret[2])==0:
        lc_p_neigh_ret[0]= -1
        lc_p_neigh_ret[1]= -1
        lc_p_neigh_ret[2]= -1
    #print("lc_p_neigh_ret=", tuple(lc_p_neigh_ret))
    #print("lc_p_neigh_ret=",lc_p_neigh_ret[0], lc_p_neigh_ret[1], lc_p_neigh_ret[2])

    p_dc_ret = np.where(dc == p_act)
    lc_p_dc_ret=list(p_dc_ret)                             # list conversion
    if len(lc_p_dc_ret[0]) == 0 and len(lc_p_dc_ret[1]) == 0 and len(lc_p_dc_ret[2]) == 0:
        lc_p_dc_ret[0]= -1
        lc_p_dc_ret[1]= -1
        lc_p_dc_ret[2]= -1

    self_ret= np.where(self == action)   # 자국에 대해 검사
    #print("self_ret=", self_ret)
    #print("self_ret=", self_ret[0], self_ret[1])        # [0]는 2차원 원소 번호, [1]은 1차원 원소 번호
    lc_self_ret=list(self_ret)                              # list conversion
    if len(lc_self_ret[0])==0 and len(lc_self_ret[1])==0:
        lc_self_ret[0]= -1
        lc_self_ret[1]= -1
    #print("lc_self_ret=", tuple(lc_self_ret))
    #print("lc_self_ret=",lc_self_ret[0],lc_self_ret[1])

    neigh_ret= np.where(neigh==action)  # 인접국에 대해 검사
    #print("neigh_ret=", neigh_ret[0], neigh_ret[1], neigh_ret[2])
    #print("neigh_ret=",neigh_ret[0], neigh_ret[1])
    lc_neigh_ret=list(neigh_ret)
    if len(lc_neigh_ret[0])==0 and len(lc_neigh_ret[1])==0 and len(lc_neigh_ret[2])==0:
        lc_neigh_ret[0]= -1
        lc_neigh_ret[1]= -1
        lc_neigh_ret[2]= -1
    #print("lc_neigh_ret=", tuple(lc_neigh_ret))
    #print("lc_neigh_ret=", lc_neigh_ret[0], lc_neigh_ret[1], lc_neigh_ret[2])

    dc_ret = np.where(dc==action)                        # DC
    lc_dc_ret=list(dc_ret)
    if len(lc_dc_ret[0])==0 and len(lc_dc_ret[1])==0 and len(lc_dc_ret[2])==0:
        lc_dc_ret[0]= -1
        lc_dc_ret[1]= -1
        lc_dc_ret[2]= -1

    return lc_p_self_ret, lc_p_neigh_ret, lc_p_dc_ret, lc_self_ret, lc_neigh_ret, lc_dc_ret

In [ ]:
# 위 함수 시험 용 --> 시험 후에 삭제할 것
edge_seq_id = 2
total_svr_list = np.arange(total_svr_no, dtype=int)
local_svr_list = np.arange(local_svr_no, dtype=int)
self_svr_list= np.arange(local_svr_no//edge_no*(edge_seq_id - 1),local_svr_no//edge_no * edge_seq_id, dtype='i')
print(self_svr_list)

self = self_svr_list.reshape(tor_no, svr_no)
neigh_svr_list= np.delete(local_svr_list, self_svr_list)
dc_svr_list = np.arange(local_svr_no, total_svr_no)
neigh = neigh_svr_list.reshape(edge_no-1, tor_no, svr_no)    # 3차원으로 수정, neigh는 전체 스위치(edge_no) 수에서 self가 사용하는 것을 제외하므로
dc = dc_svr_list.reshape(dc_edge_no,dc_tor_no, dc_svr_no)
print("self=", self)
print("neigh=", neigh)
print("dc=", dc)

p_self_ret= np.where(self ==10)  # self 리스트에서 10에 해당하는 인덱스를 반환함.
print("p_self_ret=", p_self_ret)           # (array([0]), array([1]))
print("p_self_ret=", p_self_ret[0], p_self_ret[1])  # 값만 출력하면 [0][1]  --> 즉, 2차원 self 리스트에서 값 10의 위치를 반환함

lc_p_self_ret = list(p_self_ret)
print(lc_p_self_ret)

[ 9 10 11 12 13 14 15 16 17]
self= [[ 9 10 11]
 [12 13 14]
 [15 16 17]]
neigh= [[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]]

 [[18 19 20]
  [21 22 23]
  [24 25 26]]]
dc= [[[27 28 29 30 31 32]
  [33 34 35 36 37 38]
  [39 40 41 42 43 44]]

 [[45 46 47 48 49 50]
  [51 52 53 54 55 56]
  [57 58 59 60 61 62]]]
p_self_ret= (array([0]), array([1]))
p_self_ret= [0] [1]
[array([0]), array([1])]


In [ ]:
# VNF를 순차 처리하고 처리가 끝나면 빠져나오는 함수, 상태와 행동의 히스토리를 출력하도록 수정됨.
''' Numpy 배열은 immutable 이며, 항목은 변경 가능하지만 삭제할 수는 없다. 따라서 일반 배열로 sfc_req 배열을 정의 한다 '''

def goal_sfc_ret_s_a_Q(Q, epsilon, eta, gamma, pi):

    svr_states_clone = copy.deepcopy(total_svr_states)

    actions = np.arange(total_svr_no)     # --> [0,1,2,3.....n]
    #print("act=", actions)

    #print("---------------------  start of goal_sfc_ret_s_a() --------------------------")

    #file=open("C:/users/YOO/sfc_req.txt", 'r')             # 파일에서 하나의 sfc-req를 읽어 들인다.   ################## 파일로 디렉토리와 파일 이름을 수정할 것
    #line=file.readline()

    '''---------- 시험용 sfc_req,,,,,, 여러 개를 구성하는 경우 파일로 저장해 둘 것. 파일에서 읽는 경우에는 string을 list로 변환 필요 -----------'''
    sfc_req = [10, 2, [3, 4, 4, 5, 7, 4, 2]]
    #sfc_req = [10, 2, [4,4,4,4,4]]

    #sfc_req = ast.literal_eval(line)                         # 파일에서 읽어 오는 경우 String을 list로 변환함
    #print("sfc_req=",sfc_req)

    '''---------------- sfc_req에서 정보 추출---------------'''
    L=0
    r=0

    t=len(sfc_req)

    latency = sfc_req[0]                     # 지연 값 추출 저장(추후 사용 예정)

    edge_seq_id = sfc_req[1]                 # edge_seq(엣지 스위치 번호) 추출

    total_svr_list=np.arange(total_svr_no, dtype=int)
    local_svr_list = np.arange(local_svr_no, dtype=int)
    #print("서버 번호 범위 참조 리스트=", local_svr_list)      # --> [ 0  1  2  3  4  ... 25 26]


    ''' 전체 edge 서버 갯수를 edge_no로 나눈 값(몫)에 edge-seq 번호를 이용하여 자국 서버 번호 범위를 추출한다'''
    self_svr_list= np.arange(local_svr_no//edge_no*(edge_seq_id - 1),local_svr_no//edge_no * edge_seq_id, dtype='i')
    #print("자국 서버 번호 리스트=", self_svr_list)             # --> [ 9 10 11 12 13 14 15 16 17]

    '''전체 서버 번호 행렬에서 자국 서버 번호 범위를 빼면, 결과가 1차원 list로 출력됨'''
    neigh_svr_list= np.delete(local_svr_list, self_svr_list)
    #print("인접국 서버 번호 리스트=", neigh_svr_list)          # --> [ 0  1  2  3  4  5  6  7  8 18 19 20 21 22 23 24 25 26]

    dc_svr_list=np.arange(local_svr_no, total_svr_no)
    #print(" DC 서버 번호 리스트=", dc_svr_list)                # --> [27 28 29 30 31 32....85 86]

    #------------------------------------------ end of 참조 정보 계산--------------------------------------------------------#

    ### 예: sfc_req x = [10, 1, [4, 1, 3, …]]  vnf_type_no = 0, 1, 3 ....

    sfc_req.pop(0)                                             # sfc_req[]에서 앞의 2개 원소를 삭제
    sfc_req.pop(0)
    #print("sfc_req_pop=", sfc_req[-1])

    s_a_history = [[sfc_req[0][0], np.nan]]
    # print("hist1=", s_a_history)                             # --> hist1= [[4, nan]]   -->  4 는 sfc_reg 중에서 첫번째 vnf-type 번호임.

    for aa in range(0, len(sfc_req[-1])):                      # 각 sfc_req의 single list[4, 1, 3...]에 대해서

        vnf_type_no = sfc_req[-1][aa]                           # sfc_req에서 vnf 번호를 포함한 single list를 추출하고 순서대로 대입,
        [cpu, mem] = vnf_type[vnf_type_no]                      # vnf_type[번호]별로 자원 요구량 추출

        #-----------------------------------------무작위 action의 선택 -------------------------------------------------#
        ''' pi_0로부터 vnf_type 번호와 자국, 인접국, DC 서버 범위에 해당하는 확률 행렬을 pi_1, pi_3 및 pi_5에 카피한다'''
        pi_1 = pi_0[vnf_type_no, self_svr_list[0]:self_svr_list[-1]+1].copy()   # 자국 서버들의 확률

        pi_svr_to_delete = np.append(self_svr_list, dc_svr_list)    # pi_3를 구할 때 사용
        pi_3 = np.delete(pi_0[vnf_type_no],pi_svr_to_delete)        # 인접국 서버들의 확률
        pi_5 = np.delete(pi_0[vnf_type_no],local_svr_list)          # DC 서버들의 확률

        #print("pi_1", pi_1)                                     # --> [0.01149425 0.01149425 ....0.01149425 0.01149425] --> 자국 서버 갯수만큼
        #print("pi_3", pi_3)                                     # --> [0.01149425 0.01149425 0.01149425 ...... 0.01149425 0.01149425 ]  ---> 인접국 서버 개수
        #print("pi_5", pi_5)                                     # --> [0.01149425 0.01149425 0.01149425 ..............0.01149425 0.01149425 0.01149425  }  --> DC 서버 개수

        pi_2 = np.zeros(len(self_svr_list))
        pi_4 = np.zeros(len(neigh_svr_list))
        pi_6 = np.zeros(len(dc_svr_list))

        #print("자국 서버에 설치 시도")
        ''' pi_1 행렬의 확률의 합이 1.0 이되도록 변환하여 그 결과를 pi_2에 저장한다. '''
        x = len(self_svr_list)
        if np.sum(pi_1[:]) != 0:   # ???
            for i in range(0, x):
                #pi_2[i]= pi_1[i]/np.sum(pi_1[:])    # 0으로 나누는 경우 경고: RuntimeWarning: invalid value encountered in double_scalars
                pi_2[i]= pi_1[i]/kahansum(pi_1[:])
            #print("pi_2=", pi_2)

            #print("argmax=",np.nanargmax(Q[self_svr_list[0]:self_svr_list[-1]+1]))

            if np.random.rand() < epsilon:
                action = np.random.choice(self_svr_list, p = pi_2[:])  # rand 확률 ε 보다 작으면 자국 서버 리스트(1차원)에서 자국에 대한 확률 p를 참고하여 서버를 random 선택함, 확률이 놈은 것만 선택되지 않음
                #print("OK1 action=",action)
            else:
                action = self_svr_list[np.nanargmax(Q[sfc_req[-1][aa], self_svr_list])]     # 현재의 상태(vnf_type_no)에서 Q값이 최대가 되는 행동을 선택함
                # 앞에서 인용.... 참고 --> Q = np.random.rand(a, b) * theta_0     # a x b는 행렬 Theta_0의 shape,  Uniform 분포로 0.0~1.0 값의  a x b 행렬을 생성. theta_0는 nan을 반영함
                #action = self_svr_list[np.nanargmax(Q[sfc_req[-1][aa], self_svr_list[0]:self_svr_list[-1]+1])]     # 현재의 상태(vnf_type_no)에서 Q값이 최대가 되는 행동을 선택함''' np.nanargmax는 Q 행렬에서 self_svr_list에 해당하는 슬라이스의 index 번호를 리턴하므로 actions 대신에 self_svr_list 중 하나의 서버 번호를 택하도록 수정했음 '''

            ''' 선택된 action에 따라 하나의 서버를 선택하고 설치 시도한다 --> 즉, svr_states를 갱신함'''
            if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

               # r=7    #########################

            else:
                #print("1 자국 서버 선택 불가 -->  인접국 서버를 선택 시도한다")
                #---------------------------------------------------------------------------------------
                ''' pi_3 행렬의 확률의 합이 1.0 이되도록 변환하여 그 결과를 pi_4에 저장한다. '''
                y = len(neigh_svr_list)
                if np.sum(pi_3[:]) != 0:
                    for i in range(0, y):
                        #pi_4[i]= pi_3[i]/np.sum(pi_3[:])    # 0으로 나누는 경우 경고: RuntimeWarning: invalid value encountered in double_scalars
                        pi_4[i]= pi_1[3]/kahansum(pi_3[:])
                    if np.random.rand() < epsilon:
                        action = np.random.choice(neigh_svr_list, p = pi_4[:])  # 확률 ε로 무작위 행동을 선택함
                    else:
                        ''' index=self_svr_list 및 dc_svr_list에 해당하는 원소를 Q에서 제외하고 '''
                        svr_to_delete=np.append(self_svr_list,dc_svr_list, axis=0)
                        #print("svr_to_delete=",svr_to_delete)
                        #print("118 =", Q[sfc_req[-1][aa],:])
                        action = neigh_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa],:], svr_to_delete))]

                    ''' 선택된 action에 따라 하나의 인접국 서버를 선택하고 설치 시도한다'''
                    if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                        svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                        svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                        #r=1    ###################

                    else:
                        print("1--자국 및 인접국 설치에 모두 실패했음. 데이터센터에 설치")
                        z = len(dc_svr_list)
                        if np.sum(pi_5[:]) != 0:
                            for i in range(0, z):
                                #pi_6[i]= pi_5[i]/np.sum(pi_5[:])    # random.choice 사용 시 확률의 합은 1.0이어야 함 && 0으로 나누는 경우 경고를 피하기 위해서
                                pi_6[i]= pi_1[5]/kahansum(pi_5[:])
                            if np.random.rand() < epsilon:
                                action = np.random.choice(dc_svr_list, p = pi_6[:])
                            else:
                                ''' index=local_svr_list에 해당하는 원소를 Q에서 제외한 나머지에서 최대값에 해당하는 서버를 선택 '''
                                action = dc_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa], :],local_svr_list))]
                                #print("dc_action=", action)

                            if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1

                                svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                                svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                                #r=0
                            else:
                                print("선택한 DC 서버에 자원 부족으로 설치 실패, DC 서버를 다시 random 선택하라")
                                action = np.random.choice(dc_svr_list)
                                if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                                    svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                                    svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                                    #r=0
                        else:
                            print("설치 실패, 데이터센터에 가용한 서버 자원이 없음")

                else:
                    print("2--자국 및 인접국 설치에 실패. 데이터센터에 설치함")
                    p = len(dc_svr_list)
                    if np.sum(pi_5[:]) != 0:
                        for i in range(0, p):
                            #pi_6[i]= pi_5[i]/np.sum(pi_5[:])    # random.choice 사용 시 확률의 합은 1.0이어야 함 && 0으로 나누는 경우 경고를 피하기 위해서
                            pi_6[i]= pi_1[5]/kahansum(pi_5[:])
                        if np.random.rand() < epsilon:
                            action = np.random.choice(dc_svr_list, p = pi_6[:])
                        else:
                            ''' index=local_svr_list에 해당하는 원소를 Q에서 제외한 나머지에서 최대값에 해당하는 서버를 선택 '''
                            action = dc_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa], :],local_svr_list))]
                            #print("dc_action=", action)

                        if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                            svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                            svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                            #r=0
                        else:
                            print("선택한 DC 서버에 자원 부족으로 설치 실패, DC 서버를 다시 random 선택하라")
                            action = np.random.choice(dc_svr_list)
                            if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                                svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                                svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                                #r=0
                    else:
                        print("설치 실패, 데이터센터에 가용한 서버 자원이 없음")


        else:
            #print("2 자국 서버 선택 불가 -->  인접 서버를 선택 시도한다")
            #---------------------------------------------------------------------------------------
            ''' pi_3 행렬의 확률의 합이 1.0 이되도록 변환하여 그 결과를 pi_4에 저장한다. '''
            q = len(neigh_svr_list)
            if np.sum(pi_3[:]) != 0:
                for i in range(0, q):
                    #pi_4[i]= pi_3[i]/np.sum(pi_3[:])    # 0으로 나누는 경우 경고: RuntimeWarning: invalid value encountered in double_scalars
                    pi_4[i]= pi_1[3]/kahansum(pi_3[:])
                if np.random.rand() < epsilon:
                    action = np.random.choice(neigh_svr_list, p = pi_4[:])  # 확률 ε로 무작위 행동을 선택함
                else:
                    ''' index=self_svr_list 및 dc_svr_list에 해당하는 원소를 Q에서 제외하고 '''
                    svr_to_delete=np.append(self_svr_list,dc_svr_list, axis=0)
                    #print("svr_to_delete=",svr_to_delete)
                    action = neigh_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa],:], svr_to_delete))]
                    #print("neigh_action=", action)

                ''' 선택된 action에 따라 하나의 인접국 서버를 선택하고 설치 시도한다'''
                if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                    svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                    svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                    #r=1

                else:
                    print("3--자국 및 인접국 설치에 실패했음. 데이터센터에 설치")
                    r = len(dc_svr_list)
                    if np.sum(pi_5[:]) != 0:
                        for i in range(0, r):
                            #pi_6[i]= pi_5[i]/np.sum(pi_5[:])    # random.choice 사용 시 확률의 합은 1.0이어야 함 && 0으로 나누는 경우 경고를 피하기 위해서
                            pi_6[i]= pi_1[5]/kahansum(pi_5[:])
                        if np.random.rand() < epsilon:
                            action = np.random.choice(dc_svr_list, p = pi_6[:])
                        else:
                            ''' index=local_svr_list에 해당하는 원소를 Q에서 제외한 나머지에서 최대값에 해당하는 서버를 선택 '''
                            action = dc_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa], :],local_svr_list))]
                            #print("dc_action=", action)

                        if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                            svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                            svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                            #r=0

                        else:
                            print("선택한 DC 서버에 자원 부족으로 설치 실패, DC 서버를 다시 선택하라")
                            action = np.random.choice(dc_svr_list)
                            if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                                svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                                svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]

                                #r=0

                    else:
                        print("설치 실패, 데이터센터에 가용한 서버 자원이 없음")

            else:
                print("4--자국 및 인접국 설치에 실패했음. 데이터센터에 설치")
                n = len(dc_svr_list)
                if np.sum(pi_5[:]) != 0:
                    for i in range(0, n):
                        pi_6[i]= pi_5[i]/np.sum(pi_5[:])    # random.choice 사용 시 확률의 합은 1.0이어야 함 && 0으로 나누는 경우 경고를 피하기 위해서
                        #pi_6[i]= pi_1[5]/kahansum(pi_5[:])
                    if np.random.rand() < epsilon:
                        action = np.random.choice(dc_svr_list, p = pi_6[:])
                    else:
                        ''' index=local_svr_list에 해당하는 원소를 Q에서 제외한 나머지에서 최대값에 해당하는 서버를 선택 '''
                        action = dc_svr_list[np.nanargmax(np.delete(Q[sfc_req[-1][aa], :],local_svr_list))]
                        #print("dc_action=", action)

                    if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1

                        svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                        svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]
                        #r=0

                    else:
                        print("선택한 DC 서버에 자원 부족으로 설치 실패, DC 서버를 다시 선택하라")
                        action = np.random.choice(dc_svr_list)
                        if svr_states_clone[action][0] - cpu >= 0 and svr_states_clone[action][1] -mem >= 0:   #action은 선택된 서버 번호+1
                            svr_states_clone[action][0] = svr_states_clone[action][0]-[cpu]
                            svr_states_clone[action][1] = svr_states_clone[action][1]-[mem]
                            #r=0
                else:
                    print("설치 실패, 데이터센터에 가용한 서버 자원이 없음")

        '''-------------- s_a_history appending --------------------------------'''

        s_a_history[-1][1] = action
        #a = action

        if aa != len(sfc_req[-1]) -1:       # sfc_req의 끝 vnf_type이 아니면....sfc_req에서 s, s_next를 구하여 s_a_history를 만든다
            s = sfc_req[-1][aa]
            s_next = sfc_req[-1][aa+1]
            #print("aa=", aa, "state=", s, "next=", s_next)
        else:                               # sfc의 끝을 처리하고 있으면...
            s = sfc_req[-1][aa]
            s_next = 8888

        s_a_history.append([s_next, np.nan])

        #print("s_a_hist=", s_a_history)

        '''======================  SFC 설정 결과의 가공, L (= 노드 지연) 값 추가 ========================'''

        '''---------------------if에 해당하는 경우가 없음. 삭제------------------------'''
       # if len(s_a_history) == 1:     #  s_a_history가  최초 리스트인 [n, nan] 이면
       #     s_a_history[-1][1] = action
       #     s_a_history.append([s_next, np.nan])
        '''-----------------------------------------------------------------------------'''
        if len(s_a_history) >= 3:              #  리스트의 길이가 3 이상일 때에 p_act와 act가 동시 존재한다.
            p_act = s_a_history[-3][1]         # 과거 action을 추출한다, 그리고 자국과 인접국에 대한index를 추출한다.

            '''---------- svr_loc() 함수 호출하여 서버의 위치 정보를 파악하고  L 값을 누적 계산한다 -------------'''
            lc_p_self_ret, lc_p_neigh_ret, lc_p_dc_ret, lc_self_ret, lc_neigh_ret, lc_dc_ret = svr_loc(edge_seq_id, p_act, action)


            ''' 이전 action이 자국 서버 선택인 경우'''
            #print("self- p_act, action=", p_act, action)   #####

            if lc_p_self_ret[0] != -1 and lc_self_ret[0] != -1:       # (1)번 , 즉 자국 번호 내에  p_act와 act가 있으며,,,,

                ''' 이전 action과 현재 action이 같은 서버이면 노드 지연은 없음 '''
                if p_act == action:
                    #r=10    ##############
                    pass

                ''' 이전 action과 현재 action이 같은 tor에 속하는 서버 중 하나이면'''
                if lc_p_self_ret[0] == lc_self_ret[0] and lc_p_self_ret[1] != lc_self_ret[1]:
                    L=L+1
                    #r=7
                ''' 이전 action과 현재 action이 자국 내 다른 tor에 속하면'''
                if lc_p_self_ret[0] != lc_self_ret[0]:
                    L=L+3
                    #r=3
            '''-----------------------------------------------------------'''

            ''' 이전 action이 자국 서버 선택, 현재 action이 인접국 서버의 하나이면'''
            if lc_p_self_ret[0] != -1 and lc_neigh_ret[0] != -1:
                L=L+7
            '''-------------------------------------------------------------'''

            ''' 이전 action이 인접국 서버 선택, 현재 action이 인접국 서버 선택인 경우'''
            if lc_p_neigh_ret[0] != -1 and lc_neigh_ret[0] != -1:
                ''' 이전 action과 현재 action이 같은 서버이면 노드 지연은 없음'''
                if p_act == action:
                    #r=1
                    pass
                ''' 이전 action과 현재 action이 같은 인접국(edge_no)의 같은 tor에 속하는 다른 서버이면'''
                if lc_p_neigh_ret[0] == lc_neigh_ret[0] and lc_p_neigh_ret[1] == lc_neigh_ret[1] and lc_p_neigh_ret[2] != lc_neigh_ret[2]:
                    L=L+1

                ''' 이전 action과 현재 action이 같은 인접국(edge_no) 내의 다른 tor에 속하면'''
                if lc_p_neigh_ret[0] == lc_neigh_ret[0] and lc_p_neigh_ret[1] != lc_neigh_ret[1]:
                    L=L+3

                ''' 이전 action과 현재 action이 서로 다른 인접국에 속하는 경우'''
                if lc_p_neigh_ret[0] != lc_neigh_ret[0]:
                    L=L+7
            '''-------------------------------------------------------------------'''

            ''' 이전 action이 인접국 서버 선택, 현재 action이 자국 서버 선택이면'''
            if lc_p_neigh_ret[0] != -1 and lc_self_ret[0] != -1:
                L=L+7
            '''--------------------------------------------------------------------'''

            '''이전 action이 DC 설치 이고 현재 action도  같은 DC 설치인 경우(ppt, 5번)'''

            if len(s_a_history) == 3 and lc_p_dc_ret[0]!= -1 and lc_dc_ret[0]!= -1: # 최초 action이 DC 설치일때 가입자로부터의 지연(7)을 추가
                if p_act == action:
                    L=L+7
                ''' 이전 action과 현재 action이 같은 DC의 같은 edge와 같은 tor에 속하는 다른 서버이면'''
                if lc_p_dc_ret[0] == lc_dc_ret[0] and lc_p_dc_ret[1]==lc_dc_ret[1] and lc_p_dc_ret[2] !=lc_dc_ret[2]:
                    L=L+1+7
                ''' 이전 action과 현재 action이 같은 DC의 같은 edge의 다른 tor에 속하는 다른 서버이면'''
                if lc_p_dc_ret[0] == lc_dc_ret[0] and lc_p_dc_ret[1] !=lc_dc_ret[1]:
                    L=L+3+7
                ''' 이전 action과 현재 action이 같은 DC의 다른 edge에 속하는 서버이면'''
                if lc_p_dc_ret[0] != lc_dc_ret[0]:
                    L=L+7+7

            if len(s_a_history) > 3 and lc_p_dc_ret[0]!= -1 and lc_dc_ret[0]!= -1 :
                if p_act == action:
                    pass
                ''' 이전 action과 현재 action이 같은 DC의 같은 edge와 같은 tor에 속하는 다른 서버이면'''
                if lc_p_dc_ret[0] == lc_dc_ret[0] and lc_p_dc_ret[1]==lc_dc_ret[1] and lc_p_dc_ret[2] !=lc_dc_ret[2]:
                    L=L+1
                ''' 이전 action과 현재 action이 같은 DC의 같은 edge의 다른 tor에 속하는 다른 서버이면'''
                if lc_p_dc_ret[0] == lc_dc_ret[0] and lc_p_dc_ret[1] !=lc_dc_ret[1]:
                    L=L+3
                ''' 이전 action과 현재 action이 같은 DC의 다른 edge에 속하는 서버이면'''
                if lc_p_dc_ret[0] != lc_dc_ret[0]:
                    L=L+7
            '''이전 action이 자국 또는 인접국  설치 이고 현재 action이 DC 설치인 경우'''
            if (lc_p_self_ret[0] != -1 and lc_dc_ret[0] != -1) or (lc_p_neigh_ret[0] != -1 and lc_dc_ret[0]!= -1):
                L=L+9
            '''이전 action이 DC  설치 이고 현재 action이 자국 또는 인접국 설치인 경우'''
            if (lc_p_dc_ret[0] != -1 and lc_self_ret[0] != -1) or (lc_p_dc_ret[0] != -1 and lc_neigh_ret[0]!= -1):
                L=L+9

    ''' ------------------------- SFC의 자원 요구량 계산 --------------------------------'''
    cpu_req=0
    mem_req=0
    for i in range(0, len(sfc_req[-1])):
        cpu_req=cpu_req + vnf_type[sfc_req[-1][i]][0]
        mem_req=mem_req + vnf_type[sfc_req[-1][i]][1]
    print("cpu_req mem_req=", cpu_req, mem_req)
    rsc_ratio= max([cpu_req, mem_req])/max([cpu_no, mem_no])  # resource ratio   reward 계산에 필요한 자원 사용율

    '''=----------------------- Reward 부여하고 Q 함수를 실행한다-------------------------------='''
    '''----- 다시 s_a_history에서 Q 함수에 넘겨줄 값(s, a, s_next)을 추출하고, 누적된 L 값으로 구한 reward와 함께 Q함수를 실행한다-----'''
    '''----- 즉, 하나의 SFC를 구성하는 모든 s, a, a_next에 동일한 reward를 적용한다'''
    ''' s_a_history 예==[[1, 15], [2, 14], [3, 15], [2, 10], [3, 15], [8888, nan]] '''

    hist_len = len(s_a_history)-1
    for i in range(hist_len):  # 맨끝에 [8888, nan]이 추가되었므로 -1을 취함
        s= s_a_history[i][0]              # s, a, s_next를 추출하여 하나의 SFC를 구성하는 (s, a,)에 동일한 reward를 부여하한다.
        a= s_a_history[i][1]
        s_next=s_a_history[i+1][0]

        if rsc_ratio <= 1.0 :             # 서버의 자원량 대비 SFC 자원 요구량의 배율 및
            if L <= hist_len -1:           # SFC내 vnf의 갯수를 기준으로 reward 를 결정한다.(시험하여 값을 수정해야 함)
                r= 50
            elif L <= hist_len * 1.2:
                r= 15
            elif L <= hist_len *1.5:
                r=4
            elif L <= hist_len *2:
                r= 1.5
            elif L <= hist_len*2.5:
                r= 1
            else:
                r=0

        elif rsc_ratio <= 1.4:
            if L <= hist_len * 1.0:
                r= 50
            elif L <= hist_len *1.2:
                r=15
            elif L <= hist_len *1.5:
                r= 4
            elif L <= hist_len*2.0:
                r= 1.5
            elif L <= hist_len *2.5:
                r=1
            else:
                r=0

        elif rsc_ratio <= 2.0:
            if L <= hist_len * 1.2:
                r= 50
            elif L <= hist_len *1.5:
                r=15
            elif L <= hist_len *2.0:
                r= 4
            elif L <= hist_len*2.5:
                r= 1.5
            elif L <= hist_len *3.0:
                r=1
            else:
                r=0

        else:
            if L <= hist_len *1.2:
                r=50
            elif L <= hist_len *1.5:
                r= 15
            elif L <= hist_len*2.0:
                r= 4
            elif L <= hist_len *2.5:
                r=1
            else:
                r=0

        Q = Q_learning(s, a, r, s_next, Q, eta, gamma)    #fow loop 안에서 (s,a, s_next)에 대해실행


    print("(state, action)의 history 와 Latency=", s_a_history, L)

    return s_a_history, Q, L, latency


In [ ]:
# Q러닝 알고리즘으로 SFC 설치하기
import time

start = time.perf_counter()   # 학습 시간 출력 용

eta = 0.05  # 학습률
gamma = 0.9  # 시간할인율
epsilon = 0.5  # ε-greedy 알고리즘 epsilon 초깃값, epsilog 값으로 random 선택을 함.
v = np.nanmax(Q, axis=1)  # 각 상태마다 가치의 최댓값을 계산
is_continue = True
episode = 1

L_list=[]     #  return되는 L 값을 저장할 리스트 생성
max_num=-1
min_num=-1
avg=-1

V = []  # 에피소드 별로 상태가치를 저장  --> 그래픽 처리용이므로 삭제해도 됨
V.append(np.nanmax(Q, axis=1))  # 상태 별로 행동가치의 최댓값을 계산(8x actions --> 8차원 vector로 출력됨)
#print("V=", V)

while is_continue:  # 아래에 설정한 episode의 수 만큼 반복
    print("**************** 에피소드: " + str(episode),"******************")

    # ε 값을 조금씩 감소시킴
    epsilon = epsilon / 1.05   #  2

    # goal_sfc_ret_s_a_Q() 함수 실행 후, 결과로 나온 행동 히스토리와 Q값을 변수에 저장
    s_a_history, Q, L, latency = goal_sfc_ret_s_a_Q(Q, epsilon, eta, gamma, pi_0)

    '''---------- L 값을 리스트에 append 한다'''
    L_list.append(L)
    print("L_list=", L_list)

    #if epsilon > 0.1:         # 무작위 선택을 하는 동안 min, max를 구한다.
     #   max_num = max(L_list)
     #   min_num = min(L_list)
     #   avg = np.mean(L_list)
     #   print("Min Max Avg Latency = ",min_num, max_num, avg)

    '''-----------------------------------'''
    #    reward(1,1)

    # 상태가치의 변화
    new_v = np.nanmax(Q, axis=1)  # 각 상태마다 행동가치의 최댓값을 계산
    print(np.sum(np.abs(new_v - v)))  # 상태가치 함수의 변화를 출력
    v = new_v
    V.append(v)  # 현재 에피소드가 끝난 시점의 상태가치 함수를 추가

    #print("v=", v)
    print("SFC에 설치한 VNF 수는 " + str(len(s_a_history) - 1) + " 개이고, 노드 지연은", L, "이다")

    # 에피소드 반복
    episode = episode + 1
    if episode > 100:    # 100
        break

end = time.perf_counter()   # 학습 시간 출력
print("")
print("Q-러닝 실행 시간 =", end - start, "seconds")

'''------ latency를 이용한 종료 조건 설정 --------'''
'''action의 선택은 epsilon 값에 따라 무작위로 결정되기도 하므로 이에 대응하여 latency를 사용함'''

'''------------------- End of SFC placement and deployment Program ------------------------ '''

**************** 에피소드: 1 ******************
cpu_req mem_req= 20 29
(state, action)의 history 와 Latency= [[3, 16], [4, 15], [4, 9], [5, 13], [7, 14], [4, 15], [2, 14], [8888, nan]] 14
L_list= [14]
9.318547802181001
SFC에 설치한 VNF 수는 7 개이고, 노드 지연은 14 이다
**************** 에피소드: 2 ******************
cpu_req mem_req= 20 29
(state, action)의 history 와 Latency= [[3, 17], [4, 15], [4, 12], [5, 12], [7, 16], [4, 16], [2, 10], [8888, nan]] 10
L_list= [14, 10]
5.116001156682842
SFC에 설치한 VNF 수는 7 개이고, 노드 지연은 10 이다
**************** 에피소드: 3 ******************
cpu_req mem_req= 20 29
(state, action)의 history 와 Latency= [[3, 16], [4, 15], [4, 15], [5, 16], [7, 16], [4, 15], [2, 14], [8888, nan]] 6
L_list= [14, 10, 6]
2.4583626370130105
SFC에 설치한 VNF 수는 7 개이고, 노드 지연은 6 이다
**************** 에피소드: 4 ******************
cpu_req mem_req= 20 29
(state, action)의 history 와 Latency= [[3, 16], [4, 15], [4, 15], [5, 16], [7, 16], [4, 15], [2, 14], [8888, nan]] 6
L_list= [14, 10, 6, 6]
2.4224164030975146
SFC에 설치한 VNF 수는 7

'------------------- End of SFC placement and deployment Program ------------------------ '

In [ ]:
print(Q)